In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## 10 random nodes

In [5]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [6]:
random_ls = [10,20,30,40,50,60,70,80,90,100]
random_node_analysis ={
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100


In [7]:
random_node_analysis

{'ga_vrp': [{'solution': [[74, 2, 18, 32, 36, 80, 66, 46],
    [51, 29, 38, 48, 10, 89],
    [99, 12, 6, 44, 21, 22, 57]],
   'fitness': 0.595238160270117,
   'daily_fitness': [0.6098205506391348, 0.53087102589315, 0.6083274804513742],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.561904761904762,
   'total_tarif': 116500,
   'total_duration': 125340,
   'run_time': 0.4353811740875244},
  {'solution': [[88, 36, 14, 65, 43, 10],
    [86, 6, 4, 74, 21, 44, 61, 13],
    [55, 12, 39, 45, 23]],
   'fitness': 0.6529680960845065,
   'daily_fitness': [0.6269203633734493,
    0.6525089793824049,
    0.6442479475697361],
   'days_needed': 3,
   'poi_included': 19,
   'avg_rating': 4.610526315789474,
   'total_tarif': 35000,
   'total_duration': 118860,
   'run_time': 1.137746810913086},
  {'solution': [[73, 76, 93, 9, 86, 18, 10, 4],
    [39, 34, 71, 88, 46],
    [16, 35, 7, 54, 22, 95, 1]],
   'fitness': 0.6560393530631624,
   'daily_fitness': [0.6554685228792372,
    0.6409133

In [8]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"dka_random_nodes_alldoi_1.xlsx")

### Travel Duration 1, other 0

In [7]:
random_ls = [10,20,30,40,50,60,70,80,90,100]
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100


In [8]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[44, 21, 49, 71, 48, 89, 10, 32],
    [6, 12, 22, 97, 74, 2, 36],
    [57, 80, 66, 67, 99, 18, 5, 46]],
   'fitness': 0.6446759259259259,
   'daily_fitness': [0.4740740740740741,
    0.46412037037037035,
    0.47916666666666663],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.534782608695653,
   'total_tarif': 196000,
   'total_duration': 120780,
   'run_time': 27.763322591781616},
  {'solution': [[55, 72, 44, 86, 88, 4, 17, 10, 61],
    [6, 81, 12, 90, 14, 97, 74, 13],
    [80, 28, 36, 21, 67, 45, 43, 23]],
   'fitness': 0.6719135802469136,
   'daily_fitness': [0.4849537037037037,
    0.47430555555555554,
    0.4731481481481481],
   'days_needed': 3,
   'poi_included': 25,
   'avg_rating': 4.548,
   'total_tarif': 321000,
   'total_duration': 125520,
   'run_time': 42.795228242874146},
  {'solution': [[11, 69, 10, 7, 9, 1, 53, 74, 40],
    [95, 72, 18, 88, 35, 4, 86, 46, 22],
    [76, 47, 16, 54, 71, 93, 73]],
   'fitness': 0.6701388888888

### Tarif 1, other 0

In [9]:
random_ls = [10,20,30,40,50,60,70,80,90,100]
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100


In [10]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[38, 34, 10, 74, 66, 57, 21, 36],
    [44, 18, 12, 39, 32, 89, 49],
    [6, 99, 22, 71, 46, 2]],
   'fitness': 0.8186021659565169,
   'daily_fitness': [0.6191740412979351,
    0.6007374631268436,
    0.5681415929203539],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.566666666666667,
   'total_tarif': 35000,
   'total_duration': 130320,
   'run_time': 14.670700073242188},
  {'solution': [[39, 65, 45, 74, 61, 36],
    [90, 55, 81, 14, 86, 84, 21, 43],
    [6, 12, 44, 17, 13, 10, 4, 88, 23]],
   'fitness': 0.847439126784215,
   'daily_fitness': [0.5697732997481109,
    0.599328295549958,
    0.6424433249370277],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.595652173913043,
   'total_tarif': 57000,
   'total_duration': 131880,
   'run_time': 21.580472230911255},
  {'solution': [[76, 34, 51, 18, 95, 22, 46],
    [54, 69, 39, 93, 9, 10, 73],
    [74, 40, 7, 11, 16, 1, 86, 4, 88]],
   'fitness': 0.8628094293272864,
   'daily_fit

### Rating 1, other 0

In [11]:
random_ls = [10,20,30,40,50,60,70,80,90,100]
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100


In [12]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[12, 22, 18, 74, 44, 57, 36, 10],
    [39, 64, 51, 71, 49],
    [46, 67, 48, 32, 89, 5, 66, 2, 21]],
   'fitness': 0.7598099046015714,
   'daily_fitness': [0.43333333333333357,
    0.3133333333333333,
    0.4555555555555558],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.572727272727273,
   'total_tarif': 141500,
   'total_duration': 132120,
   'run_time': 7.127506494522095},
  {'solution': [[74, 12, 6, 45, 88, 13, 36, 23],
    [81, 14, 55, 84, 72, 44, 17, 4, 61],
    [39, 86, 28, 67, 10, 21, 43]],
   'fitness': 0.8097222222222222,
   'daily_fitness': [0.5395833333333342,
    0.4370370370370362,
    0.37857142857142856],
   'days_needed': 3,
   'poi_included': 24,
   'avg_rating': 4.583333333333333,
   'total_tarif': 265000,
   'total_duration': 131700,
   'run_time': 19.910401344299316},
  {'solution': [[76, 16, 95, 18, 86, 73, 46, 7, 35, 4],
    [11, 69, 72, 9, 88, 1, 93, 40],
    [64, 34, 53, 10, 22, 74]],
   'fitness': 0.82628205128205

## 87 POIs

In [5]:
def result_to_xlsx(result,n,filename):
    df_result = pd.DataFrame()
    for i in result:
        temp = pd.DataFrame(result[i],columns=["fitness","run_time"])
        temp['n'] = n
        temp['method'] = i
        df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [9]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ga_vrp"])==0 or run_time_analysis["ga_vrp"][-1][-1] < max_runtime:
        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ts_vrp"])==0 or run_time_analysis["ts_vrp"][-1][-1] < max_runtime:
        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["sa_vrp"])==0 or run_time_analysis["sa_vrp"][-1][-1] < max_runtime:
        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["dka_vrp"])==0 or run_time_analysis["dka_vrp"][-1][-1] < max_runtime:
        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [10]:
run_time_analysis

{'acsbso_vrp': [[0.8871090534979426, 0.38243961334228516],
  [0.6521678424456201, 2.975457191467285],
  [0.6153649113966573, 2.314694881439209],
  [0.6438124632569079, 6.148261308670044],
  [0.7030226563559895, 16.660783767700195],
  [0.6924740386265574, 25.01412868499756],
  [0.6647941396343559, 17.487596035003662],
  [0.6490533283588839, 59.57031178474426],
  [0.6290064506871007, 60.35109305381775],
  [0.614246999528495, 86.24367785453796],
  [0.6055980654362337, 68.67522096633911],
  [0.608312406692506, 115.05306935310364],
  [0.602657584772457, 154.63263368606567],
  [0.587594123970585, 140.28878736495972],
  [0.5891579724634121, 274.7147500514984],
  [0.5864081337421732, 271.5579023361206],
  [0.5766677310080689, 307.7164525985718]],
 'ga_vrp': [[0.8871090534979426, 0.08224034309387207],
  [0.6514476778365665, 0.3145167827606201],
  [0.5999338624338623, 0.5032882690429688],
  [0.6348618459729566, 0.4026656150817871],
  [0.6934033147922035, 0.5346200466156006],
  [0.656039353063162

In [18]:
result_to_xlsx(run_time_analysis,n,"various_numbers_pois_all_1.xlsx")

### Travel duration 1, other 0

In [19]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ga_vrp"])==0 or run_time_analysis["ga_vrp"][-1][-1] < max_runtime:
        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ts_vrp"])==0 or run_time_analysis["ts_vrp"][-1][-1] < max_runtime:
        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["sa_vrp"])==0 or run_time_analysis["sa_vrp"][-1][-1] < max_runtime:
        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["dka_vrp"])==0 or run_time_analysis["dka_vrp"][-1][-1] < max_runtime:
        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [20]:
run_time_analysis

{'acsbso_vrp': [[0.9406635802469135, 0.36297154426574707],
  [0.8636574074074074, 1.9900388717651367],
  [0.7993827160493828, 4.128458261489868],
  [0.7623456790123456, 11.887083292007446],
  [0.7101929012345679, 18.674004793167114],
  [0.670138888888889, 35.12737441062927],
  [0.6152447089947091, 27.620591640472412],
  [0.58125, 41.799052476882935],
  [0.5596064814814815, 79.64422082901001],
  [0.5329320987654321, 81.68603873252869],
  [0.525003507295174, 118.13048696517944],
  [0.503858024691358, 136.39345693588257],
  [0.4870963912630579, 157.52197980880737],
  [0.44721671075837743, 42.57802987098694],
  [0.47424382716049385, 178.0877022743225],
  [0.46716820987654323, 223.61386108398438],
  [0.4521525117071094, 324.1364800930023]],
 'ga_vrp': [[0.9406635802469135, 0.07889795303344727],
  [0.8625771604938272, 0.37857770919799805],
  [0.7917438271604937, 0.4776465892791748],
  [0.7332561728395062, 0.35617756843566895],
  [0.6707716049382716, 0.42639827728271484],
  [0.633757716049382

In [21]:
result_to_xlsx(run_time_analysis,n,"various_numbers_pois_dwaktu_1.xlsx")

### Tarif 1, other 0

In [22]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ga_vrp"])==0 or run_time_analysis["ga_vrp"][-1][-1] < max_runtime:
        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ts_vrp"])==0 or run_time_analysis["ts_vrp"][-1][-1] < max_runtime:
        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["sa_vrp"])==0 or run_time_analysis["sa_vrp"][-1][-1] < max_runtime:
        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["dka_vrp"])==0 or run_time_analysis["dka_vrp"][-1][-1] < max_runtime:
        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [23]:
run_time_analysis

{'acsbso_vrp': [[0.9902777777777778, 0.3616502285003662],
  [0.75, 1.7270264625549316],
  [0.8065972222222222, 4.714240074157715],
  [0.8611166225749559, 3.192894458770752],
  [0.8770612874779542, 11.766361236572266],
  [0.8628094293272864, 21.301560401916504],
  [0.8225769068689424, 16.807252883911133],
  [0.7826664462081129, 19.8129620552063],
  [0.7737700983921285, 58.39465641975403],
  [0.7544646271510517, 83.08348155021667],
  [0.747884708541902, 83.20264840126038],
  [0.7293069417456672, 102.636470079422],
  [0.6776519870020105, 38.86884379386902],
  [0.7144757298324409, 153.26738810539246],
  [0.6971425786135759, 118.47205662727356],
  [0.6629766949152542, 44.01990509033203],
  [0.6839080459770115, 247.9904453754425]],
 'ga_vrp': [[0.9902777777777778, 0.11005258560180664],
  [0.75, 0.20385956764221191],
  [0.7470293209876543, 0.24768996238708496],
  [0.8611552028218695, 0.5480856895446777],
  [0.8740454144620811, 1.509838342666626],
  [0.8203168304358781, 1.1090219020843506],
  

In [24]:
result_to_xlsx(run_time_analysis,n,"various_numbers_pois_dtarif_1.xlsx")

### Rating 1, other 0

In [6]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ga_vrp"])==0 or run_time_analysis["ga_vrp"][-1][-1] < max_runtime:
        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["ts_vrp"])==0 or run_time_analysis["ts_vrp"][-1][-1] < max_runtime:
        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["sa_vrp"])==0 or run_time_analysis["sa_vrp"][-1][-1] < max_runtime:
        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["dka_vrp"])==0 or run_time_analysis["dka_vrp"][-1][-1] < max_runtime:
        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [7]:
run_time_analysis

{'acsbso_vrp': [[0.8795833333333335, 0.6249508857727051],
  [0.8637841710758376, 1.3538463115692139],
  [0.8542603615520283, 6.624521255493164],
  [0.8340225563909776, 9.540267705917358],
  [0.8639510489510489, 14.198480129241943],
  [0.8262820512820512, 23.807022094726562],
  [0.7843336216252883, 32.741103649139404],
  [0.7386900522317187, 31.018046617507935],
  [0.7164036817882968, 53.284568309783936],
  [0.674605887939221, 22.768742322921753],
  [0.6880395466602363, 110.74433636665344],
  [0.6634636582553248, 136.64147686958313],
  [0.6506414349805154, 115.47816514968872],
  [0.6482275788727401, 238.86089754104614],
  [0.6459906759906759, 273.2661461830139],
  [0.6246213774775604, 166.76018619537354],
  [0.6171626186712393, 763.3659780025482]],
 'ga_vrp': [[0.8802777777777779, 0.14566469192504883],
  [0.8642857142857141, 0.25557374954223633],
  [0.8517912257495589, 0.38154125213623047],
  [0.833250951452706, 0.5766308307647705],
  [0.842161172161172, 0.8603320121765137],
  [0.808828

In [8]:
result_to_xlsx(run_time_analysis,n,"various_numbers_pois_drating_1.xlsx")

## DOI Combination

In [51]:
def result_to_xlsx_doi(result,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        method = i
        iteration = 0
        temp_method = pd.DataFrame()
        for j in result[i]:
            j['solution'] = str(j['solution'])
            j['daily_fitness'] = str(j['daily_fitness'])
            state = random_ls[iteration]
            temp_state = pd.DataFrame(j,index=[0])
            temp_state = temp_state.T
            temp_state = temp_state.rename(columns={0:str(state)}) 
            if iteration == 0:
                temp_method = temp_state.copy()
            else:
                temp_method = temp_method.join(temp_state)
            iteration += 1
        temp_method['method'] = method
        df_result = pd.concat([df_result,temp_method])

    df_result = df_result.reset_index()
    df_result = df_result.rename(columns={'index':"metrics"})
    df_result = df_result.sort_values(['metrics','method'])
    df_result.to_excel(filename,index=False)
    return df_result

In [52]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [14]:
doi_combi

[(0.15, 0.45, 0.77),
 (0.71, 0.73, 0.43),
 (0.8, 0.53, 0.08),
 (0.46, 0.05, 0.93),
 (0.95, 0.34, 0.31),
 (0.77, 0.2, 0.18),
 (0.19, 0.35, 0.63),
 (0.96, 0.21, 0.96),
 (0.56, 0.9, 0.82),
 (0.16, 0.65, 0.12)]

In [56]:
for doi in doi_combi:
    print("doi combination = ",doi)
    random_ls = [10,20,30,40,50,60,70,80,90,100]
    random_node_analysis ={
        "acsbso_vrp": [],
        "ga_vrp": [],
        "ts_vrp": [],
        "sa_vrp": [],
        "dka_vrp": []
    }
    for i in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
        travel_days = 3
        dwaktu,dtarif,drating = doi

        print("random state = ",random_ls[i])

        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)

        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ga_vrp"].append(results_ga_vrp)

        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ts_vrp"].append(results_ts_vrp)

        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["sa_vrp"].append(results_sa_vrp)

        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["dka_vrp"].append(results_dka_vrp)
    
    filename = "random_doi_" + str(doi) +".xlsx"
    result_to_xlsx_doi(random_node_analysis,random_ls,filename)

doi combination =  (0.15, 0.45, 0.77)
random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100
doi combination =  (0.71, 0.73, 0.43)
random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100
doi combination =  (0.8, 0.53, 0.08)
random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100
doi combination =  (0.46, 0.05, 0.93)
random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100
doi combination =  (0.95, 0.34, 0.31)
random state =  10
random state =  20
random st